In [1]:
import tensorflow as tf
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
from tqdm import tqdm
import pandas as pd
import re
tqdm.pandas(desc="progress-bar")
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
SEED = 2000


train=pd.read_csv('train_E6oV3lV.csv')
test=pd.read_csv('test_tweets_anuFYb8.csv')

target=train.label
train=train.drop('label',1)
data=train.append(test)
tweet =data.tweet


# pre-processing
print("Text Preprocessing ====>")
data['tweet']=data['tweet'].apply(lambda x: x.lower())
data['tweet']=data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['tweet']=data['tweet'].apply(lambda x: x.replace('user',''))
stop = stopwords.words('english')
data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

x = data['tweet'][:len(train)]
y = target

x_train, x_validation_and_test , y_train , y_validation_and_test = train_test_split(x, y,test_size=.3,random_state= SEED)

x_validation , x_test , y_validation , y_test = train_test_split(x_validation_and_test,y_validation_and_test,test_size=0.5,random_state=SEED)


print("Train set has total {0} entries with {1:.2f} % positive, {2:.2f}% negative".format(len(x_train),(len(x_train[y_train==0])/(len(x_train)*1.))*100,
                                                                                          (len(x_train[y_train == 1]) / (len(x_train) * 1.)) * 100))



print("Validation set has total {0} entries with {1:.2f} % positive, {2:.2f}% negative".format(len(x_validation),(len(x_validation[y_validation==0])/(len(x_validation)*1.))*100,
                                                                                          (len(x_validation[y_validation == 1]) / (len(x_validation) * 1.)) * 100))




print("Test set has total {0} entries with {1:.2f} % positive, {2:.2f}% negative".format(len(x_test),(len(x_test[y_test==0])/(len(x_test)*1.))*100,
                                                                                          (len(x_test[y_test == 1]) / (len(x_test) * 1.)) * 100))





def labelize_tweets_ug(tweets,label):

    result = []
    prefix = label
    for i , t in zip(tweets.index,tweets):
        result.append(TaggedDocument(t.split(),[prefix + '_%s' % i ]))

    return result


all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x,'all')

def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n= 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1

    return vecs





/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/subir/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module wi

Text Preprocessing ====>
Train set has total 22373 entries with 92.93 % positive, 7.07% negative
Validation set has total 4794 entries with 92.89 % positive, 7.11% negative
Test set has total 4795 entries with 93.33 % positive, 6.67% negative


In [2]:
# Distributed Bag of Words

from sklearn.linear_model import LogisticRegression
cores = multiprocessing.cpu_count()

model_ug_dbow = Doc2Vec(dm=0,vector_size=100,negative=5,min_count=2,workers=cores,alpha=0.1,min_alpha=0.1)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])


for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v),epochs=1)
    model_ug_dbow.alpha -=0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation , 100)


clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)
print('score',clf.score(validation_vecs_dbow,y_validation))

100%|██████████| 31962/31962 [00:00<00:00, 2508623.75it/s]


score 0.942636629119733


In [3]:
from gensim.models.phrases import Phrases , Phraser
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
tokenized_train = [t.split() for t in all_x]
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)

In [4]:
def labelize_tweets_tg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result
  
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v_tg = labelize_tweets_tg(all_x, 'all')

In [5]:
# Distributed Memory Mean

model_tg_dmm = Doc2Vec(dm=1,dm_mean=1,vector_size=100,negative=5,min_count=2,workers=cores,alpha=0.1,min_alpha=0.1)
model_tg_dmm.build_vocab([x for x in tqdm(all_x_w2v_tg)])


for epoch in range(30):
    model_tg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg),epochs=1)
    model_tg_dmm.alpha -=0.002
    model_tg_dmm.min_alpha = model_tg_dmm.alpha

train_vecs_dmm = get_vectors(model_tg_dmm, x_train, 100)
validation_vecs_dmm = get_vectors(model_tg_dmm, x_validation , 100)


clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)
print('score',clf.score(validation_vecs_dmm,y_validation))

100%|██████████| 31962/31962 [00:00<00:00, 2682293.45it/s]


score 0.9336670838548186


In [6]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [7]:
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_tg_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
train_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_train, 200)
validation_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_validation, 200)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_vecs_ugdbow_tgdmm, y_train)

print(clf.score(train_vecs_ugdbow_tgdmm, y_train))
print(clf.score(validation_vecs_ugdbow_tgdmm, y_validation))

0.9451124122826622
0.9451397580308719


In [14]:
import numpy as np
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence



In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath="d2v_09_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max') 
callbacks_list = [checkpoint, early_stop]
np.random.seed(seed)
model_d2v_09_es = Sequential()
model_d2v_09_es.add(Dense(256, activation='relu', input_dim=200))
model_d2v_09_es.add(Dense(256, activation='relu'))
model_d2v_09_es.add(Dense(256, activation='relu'))
model_d2v_09_es.add(Dense(1, activation='sigmoid'))
model_d2v_09_es.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_09_es.fit(train_vecs_ugdbow_tgdmm, y_train,
                    validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), 
                    epochs=100, batch_size=32, verbose=2, callbacks=callbacks_list)

Train on 22373 samples, validate on 4794 samples
Epoch 1/100
 - 2s - loss: 0.1639 - acc: 0.9418 - val_loss: 0.1388 - val_acc: 0.9497

Epoch 00001: val_acc improved from -inf to 0.94973, saving model to d2v_09_best_weights.01-0.9497.hdf5
Epoch 2/100
 - 1s - loss: 0.1211 - acc: 0.9565 - val_loss: 0.1347 - val_acc: 0.9497

Epoch 00002: val_acc improved from 0.94973 to 0.94973, saving model to d2v_09_best_weights.02-0.9497.hdf5
Epoch 3/100
 - 2s - loss: 0.0933 - acc: 0.9656 - val_loss: 0.1628 - val_acc: 0.9514

Epoch 00003: val_acc improved from 0.94973 to 0.95140, saving model to d2v_09_best_weights.03-0.9514.hdf5
Epoch 4/100
 - 1s - loss: 0.0651 - acc: 0.9759 - val_loss: 0.1553 - val_acc: 0.9506

Epoch 00004: val_acc did not improve from 0.95140
Epoch 5/100
 - 2s - loss: 0.0423 - acc: 0.9844 - val_loss: 0.2123 - val_acc: 0.9510

Epoch 00005: val_acc did not improve from 0.95140
Epoch 6/100
 - 2s - loss: 0.0316 - acc: 0.9886 - val_loss: 0.2411 - val_acc: 0.9451

Epoch 00006: val_acc did n

In [16]:
from keras.models import load_model
loaded_model = load_model('d2v_09_best_weights.03-0.9514.hdf5')
loaded_model.evaluate(x=validation_vecs_ugdbow_tgdmm, y=y_validation)

4794/4794 [==============================] - 0s 39us/step


[0.16283375229632102, 0.9513975801595211]

In [18]:
pred = loaded_model.predict_classes(get_concat_vectors(model_ug_dbow,model_tg_dmm, data[len(train):], 200))

In [20]:
np.unique(pred)

array([0, 1], dtype=int32)

In [21]:
submission=pd.DataFrame()
submission['id']=test['id']
submission['label']=pred
submission.to_csv('submission_twitter_200d.csv',index=False)